In [1]:
import requests
import pandas as pd
import geopandas as gpd
import arcpy
import requests
import csv

In [2]:
# creating workspace
workspace = arcpy.env.workspace = r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb'
working_dir = r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb'

In [3]:
# define data source and where to fetch csv data
url = r"https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&year=2022&ttype=daily&quick_pick=30_d&begin_date=2022-11-22&end_date=2022-11-22"

In [4]:
# bring in data
data = requests.get(url)

In [5]:
# convert csv to pandas data frame
df = pd.read_csv(r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\table (5).csv', skiprows=[0, 1, 2, 4])
print(df)

     Station Name  Latitude  Longitude  ...  Min Temp Flag  Avg Temp  Avg Temp Flag
0             Ada  47.32119  -96.51406  ...            NaN    48.827            NaN
1             Ada  47.32119  -96.51406  ...            NaN    38.091            NaN
2             Ada  47.32119  -96.51406  ...            NaN    39.056            NaN
3             Ada  47.32119  -96.51406  ...            NaN    44.105            NaN
4             Ada  47.32119  -96.51406  ...            NaN    51.091            NaN
...           ...       ...        ...  ...            ...       ...            ...
4225      Zeeland  46.01351  -99.68768  ...            NaN     8.409            NaN
4226      Zeeland  46.01351  -99.68768  ...            NaN    12.984            NaN
4227      Zeeland  46.01351  -99.68768  ...            NaN    13.921            NaN
4228      Zeeland  46.01351  -99.68768  ...            NaN    19.584            NaN
4229      Zeeland  46.01351  -99.68768  ...            NaN    21.025        

In [6]:
# read data only on headers I am interested in
header = ['Station Name', 'Latitude', 'Longitude', 'Year', 'Month' , 'Day' , 'Max Temp', 'Min Temp', 'Avg Temp']
df=df[header]
df.head()

,Station Name,Latitude,Longitude,Year,Month,Day,Max Temp,Min Temp,Avg Temp
0,Ada,47.32119,-96.51406,2022,10,24,63.356,34.297,48.827
1,Ada,47.32119,-96.51406,2022,10,25,42.402,33.780,38.091
2,Ada,47.32119,-96.51406,2022,10,26,48.200,29.912,39.056
3,Ada,47.32119,-96.51406,2022,10,27,53.744,34.466,44.105
4,Ada,47.32119,-96.51406,2022,10,28,62.636,39.546,51.091


In [8]:
# create a new df of avg, max and min temps of each station by month
df2 = df.groupby('Station Name').agg(
    AvgtempByStation = pd.NamedAgg(column='Avg Temp', aggfunc=pd.Series.mean),
    avgMaxTempByStation = pd.NamedAgg(column='Max Temp', aggfunc=pd.Series.mean),
    avgMMinTempByStation = pd.NamedAgg(column='Min Temp', aggfunc=pd.Series.mean),
)
# https://dataindependent.com/pandas/pandas-group-by-guide-3-methods/


df2.to_csv(r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\AverageOutput1.csv')
df2.head()

,AvgtempByStation,avgMaxTempByStation,avgMMinTempByStation
Station Name,,,
Ada,31.558700,40.137067,22.980000
Adams,26.883633,35.875733,17.891000
Alamo,22.730467,32.942400,12.518167
Alvarado,30.058167,39.090700,21.025033
Amidon,27.393600,37.833633,16.953067


In [9]:
# merge the two dataframes on Station Name
df_merged = df.merge(df2, left_on='Station Name', right_on='Station Name')
df_merged.head()

,Station Name,Latitude,Longitude,Year,Month,Day,Max Temp,Min Temp,Avg Temp,AvgtempByStation,avgMaxTempByStation,avgMMinTempByStation
0,Ada,47.32119,-96.51406,2022,10,24,63.356,34.297,48.827,31.5587,40.137067,22.98
1,Ada,47.32119,-96.51406,2022,10,25,42.402,33.780,38.091,31.5587,40.137067,22.98
2,Ada,47.32119,-96.51406,2022,10,26,48.200,29.912,39.056,31.5587,40.137067,22.98
3,Ada,47.32119,-96.51406,2022,10,27,53.744,34.466,44.105,31.5587,40.137067,22.98
4,Ada,47.32119,-96.51406,2022,10,28,62.636,39.546,51.091,31.5587,40.137067,22.98


In [10]:
# Get rid of duplicate station names so just individual stations with it's unique average temp for the last 30 days
df_merged_drop_duplicates = df_merged.drop_duplicates(subset=['Station Name'])
df_merged_drop_duplicates.head(100)

,Station Name,Latitude,Longitude,Year,Month,Day,Max Temp,Min Temp,Avg Temp,AvgtempByStation,avgMaxTempByStation,avgMMinTempByStation
0,Ada,47.321190,-96.514060,2022,10,24,63.356,34.297,48.827,31.558700,40.137067,22.980000
30,Adams,48.499880,-98.075880,2022,10,24,52.880,30.522,41.701,26.883633,35.875733,17.891000
60,Alamo,48.546520,-103.471860,2022,10,24,35.161,22.343,28.752,22.730467,32.942400,12.518167
90,Alvarado,48.245942,-97.021532,2022,10,24,66.128,32.851,49.490,30.058167,39.090700,21.025033
120,Amidon,46.488440,-103.316290,2022,10,24,34.259,25.569,29.914,27.393600,37.833633,16.953067
...,...,...,...,...,...,...,...,...,...,...,...,...
2850,Ottertail,46.410950,-95.574300,2022,10,24,63.932,35.341,49.637,32.817933,40.735500,24.899900
2880,Parkers Prairie,46.169410,-95.356020,2022,10,24,63.158,37.330,50.244,33.476833,42.068567,24.884633
2910,Pekin,47.807640,-98.310670,2022,10,24,56.588,31.606,44.097,28.701067,37.890067,19.511600
2940,Perham,46.610490,-95.601880,2022,10,24,64.112,35.823,49.968,32.641333,40.935000,24.347200


In [ ]:
# export as csv
df_merged_drop_duplicates.to_csv(r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\AllAverages.csv')

In [ ]:
arcpy.conversion.TableToTable("AllAverages.csv", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb", "Allaverages_arcpy", '', 'Field1 "Field1" true true false 4 Long 0 0,First,#,AllAverages.csv,Field1,-1,-1;Station Name "Station Name" true true false 8000 Text 0 0,First,#,AllAverages.csv,Station Name,0,8000;Latitude "Latitude" true true false 8 Double 0 0,First,#,AllAverages.csv,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,AllAverages.csv,Longitude,-1,-1;Year "Year" true true false 4 Long 0 0,First,#,AllAverages.csv,Year,-1,-1;Month "Month" true true false 4 Long 0 0,First,#,AllAverages.csv,Month,-1,-1;Day "Day" true true false 4 Long 0 0,First,#,AllAverages.csv,Day,-1,-1;Max Temp "Max Temp" true true false 8 Double 0 0,First,#,AllAverages.csv,Max Temp,-1,-1;Min Temp "Min Temp" true true false 8 Double 0 0,First,#,AllAverages.csv,Min Temp,-1,-1;Avg Temp "Avg Temp" true true false 8 Double 0 0,First,#,AllAverages.csv,Avg Temp,-1,-1', '')

In [ ]:
arcpy.management.XYTableToPoint("Allaverages_arcpy", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\NDAWN_stations", "Longitude", "Latitude", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

In [ ]:
# IDW
arcpy.ga.IDW("NDAWN_stations", "Avg", "IDW_NDAWN_AVG", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\IDW_NDAWN_AVG", 0.0144790399999999, 2, "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)

In [12]:
# kriging ordinary avg
out_surface_raster = arcpy.sa.Kriging("NDAWN_stations", "Avg", "Spherical # # # #", 0.0144790399999999, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\Kriging_NDAWN_ordinary")

In [1]:
# kriging ordinary max 
out_surface_raster = arcpy.sa.Kriging("NDAWN_stations", "Max", "Spherical # # # #", 0.0144790399999999, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\Kriging_NDAWN_ordinary")

In [2]:
# kriging ordinary min
out_surface_raster = arcpy.sa.Kriging("NDAWN_stations", "Min", "Spherical # # # #", 0.0144790399999999, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\Kriging_NDAWN_ordinary")

In [11]:
# kriging universal
out_surface_raster = arcpy.sa.Kriging("NDAWN_stations", "Avg", "LinearDrift 0.014479 # # #", 0.0144790399999999, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\Kriging_NDAWN_universal")

In [13]:
# natural neighbor
out_raster = arcpy.sa.NaturalNeighbor("NDAWN_stations", "Avg", 0.0144790399999999); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\Natural_NDAWN_AVG")

In [ ]:
# dkb
arcpy.ga.DiffusionInterpolationWithBarriers("NDAWN_stations", "Avg", "DK_NDAWN", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab3\Lab3Part2\Lab3Part2.gdb\DKB_NDAWN", 0.0144790399999999, None, None, 100, None, None, None, None)

In [ ]:
# moreans I
arcpy.stats.SpatialAutocorrelation("NDAWN_stations", "Avg", None, "K_NEAREST_NEIGHBORS", "EUCLIDEAN_DISTANCE", "ROW", None, None, 8)